In [1]:
import torch as th
import syft as sy

In [2]:
sy.sandbox(globals(), False, False)

Setting up Sandbox...
Done!


In [89]:
# positive, non-negative, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[0, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 0, 0, 0]])
    )
)

y = (
    th.tensor([1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[0, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 0, 0, 0]])
    )
)

y = x > y

assert y.sensitivity == th.tensor([0]).byte()
assert y.max_vals == th.tensor([0])
assert y.min_vals == th.tensor([0])
assert (y.entities == th.tensor([[0, 0, 0, 0]]).byte()).all()



In [95]:
x.child._could_overlap_with(x.child)

tensor([[0, 0, 0, 0]], dtype=torch.uint8)

In [99]:
x.child.expanded_max_vals

tensor([[1, 1, 1, 1]])

In [100]:
x.child.expanded_min_vals

tensor([[0, 0, 0, 0]])

In [96]:
could_self_top_overlap_with_other_bottom = (
            x.child.expanded_max_vals
            < x.child.expanded_min_vals  # TODO: expand later for better efficiency
        )

In [97]:
could_self_top_overlap_with_other_bottom

tensor([[0, 0, 0, 0]], dtype=torch.uint8)

In [92]:
y.child.max_ent_conts

tensor([[0, 0, 0, 0]], dtype=torch.uint8)

In [93]:
y.child.min_ent_conts

tensor([[0, 0, 0, 0]], dtype=torch.uint8)

In [90]:
y.sensitivity

tensor([0], dtype=torch.uint8)

In [62]:

# negative, non-positive, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([-1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-1, 0, 0, 0]]), max_ent_conts=th.tensor([[0, 0, 0, 0]])
    )
)

y = -x

assert y.sensitivity == th.tensor([1])
assert y.max_vals == th.tensor([1])
assert y.min_vals == th.tensor([0])
assert (y.entities == th.tensor([[1, 0, 0, 0]])).all()

# negative, positive, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([-1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-1, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 0, 0, 0]])
    )
)

y = -x

assert y.sensitivity == th.tensor([2])
assert y.max_vals == th.tensor([1])
assert y.min_vals == th.tensor([-1])
assert (y.entities == th.tensor([[1, 0, 0, 0]])).all()

# negative, positive, dual entitiy, overlapping, symmetric mult
x = (
    th.tensor([1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-1, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 1, 0, 0]])
    )
)

y = -x

assert y.sensitivity == th.tensor([2])
assert y.max_vals == th.tensor([1])
assert y.min_vals == th.tensor([-2])
assert (y.entities == th.tensor([[1, 1, 0, 0]])).all()

# negative, positive, dual entitiy, non-overlapping, non-symmetric add
x = (
    th.tensor([1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-1, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 1, 0, 0]])
    )
)

y = (
    th.tensor([5])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[4, 0, 0, 0]]), max_ent_conts=th.tensor([[5, 5, 0, 0]])
    )
)

z = x * y

assert z.sensitivity == th.tensor([20])
assert z.max_vals == th.tensor([20])
assert z.min_vals == th.tensor([-10])
assert (z.entities == th.tensor([[1, 1, 0, 0]])).all()